## Basic Flow

In [8]:
import openai
from crewai.flow.flow import Flow, start, listen

from dotenv import load_dotenv
import os

load_dotenv()

True

In [9]:
openai_client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [19]:
class MovieRecommendationFlow(Flow):

    @start()
    def generate_genre(self):
        response = openai_client.responses.create(
            model="gpt-4o-mini",
            input=[
                {
                    "role": "user", 
                    "content": "Genearate a random movie genre"
                },
            ]
        )
        random_genre = response.output_text
        self.state["random_genre"] = random_genre
        return random_genre
    
    @listen(generate_genre)
    def recommend_movie(self, random_genre):
        response = openai_client.responses.create(
            model="gpt-4o-mini",
            input=[
                {
                    "role": "user", 
                    "content": f"Recommend a movie with the genre: {random_genre}"
                }
            ]
        )
        movie_recommendation = response.output_text
        self.state["movie_recommendation"] = movie_recommendation
        return movie_recommendation

In [20]:
flow = MovieRecommendationFlow()
final_result = await flow.kickoff_async()

print(f"\n Movie Recommendation : {final_result}")

╭─────────────────────────────── Flow Execution ───────────────────────────────╮
│                                                                              │
│  Starting Flow Execution                                                     │
│  Name: MovieRecommendationFlow                                               │
│  ID: d01eeed2-83e6-4bb5-8108-e598cd56828a                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: MovieRecommendationFlow
    ID: d01eeed2-83e6-4bb5-8108-e598cd56828a
└── 🧠 Starting Flow...

 Flow started with ID: d01eeed2-83e6-4bb5-8108-e598cd56828a
🌊 Flow: MovieRecommendationFlow
    ID: d01eeed2-83e6-4bb5-8108-e598cd56828a
├── 🧠 Starting Flow...
└── 🔄 Running: generate_genre

🌊 Flow: MovieRecommendationFlow
    ID: d01eeed2-83e6-4bb5-

In [24]:
flow.state

{'id': 'd01eeed2-83e6-4bb5-8108-e598cd56828a',
 'random_genre': 'How about a **sci-fi mystery thriller**? It could involve advanced technology, an enigmatic disappearance, and a race against time to uncover the truth.',
 'movie_recommendation': 'I recommend **"Annihilation" (2018)**. This film is a blend of sci-fi, mystery, and thriller elements. It follows a group of scientists, led by a biologist (played by Natalie Portman), who venture into a mysterious quarantined zone known as "The Shimmer," where the laws of nature don\'t apply. The film delves into themes of advanced technology and genetic mutation while unraveling the enigma behind the disappearance of her husband and the secrets hidden within the Shimmer. With atmospheric visuals and a haunting score, **"Annihilation"** presents a gripping narrative filled with tension and philosophical undertones.'}

In [25]:
from IPython.display import Markdown
Markdown(flow.state["movie_recommendation"])

I recommend **"Annihilation" (2018)**. This film is a blend of sci-fi, mystery, and thriller elements. It follows a group of scientists, led by a biologist (played by Natalie Portman), who venture into a mysterious quarantined zone known as "The Shimmer," where the laws of nature don't apply. The film delves into themes of advanced technology and genetic mutation while unraveling the enigma behind the disappearance of her husband and the secrets hidden within the Shimmer. With atmospheric visuals and a haunting score, **"Annihilation"** presents a gripping narrative filled with tension and philosophical undertones.

In [26]:
from pprint import pprint
pprint(flow.state)

{'id': 'd01eeed2-83e6-4bb5-8108-e598cd56828a',
 'movie_recommendation': 'I recommend **"Annihilation" (2018)**. This film is '
                         'a blend of sci-fi, mystery, and thriller elements. '
                         'It follows a group of scientists, led by a biologist '
                         '(played by Natalie Portman), who venture into a '
                         'mysterious quarantined zone known as "The Shimmer," '
                         "where the laws of nature don't apply. The film "
                         'delves into themes of advanced technology and '
                         'genetic mutation while unraveling the enigma behind '
                         'the disappearance of her husband and the secrets '
                         'hidden within the Shimmer. With atmospheric visuals '
                         'and a haunting score, **"Annihilation"** presents a '
                         'gripping narrative filled with tension and '
                    

## States in Flows

In [29]:
from crewai.flow.flow import Flow, listen, start

class TaskManagementFlow(Flow):

    @start()
    def generate_task(self):
        print(f"Flow started. State ID : {self.state['id']}")

        # Step 1 : Generate a task
        self.state["task"] = "Fix a critical bug in the payment system"
        self.state["status"] = "Pending"
        print(f"Task generated : {self.state['task']} (Status : {self.state['status']})")
    
    @listen(generate_task)
    def start_task(self):
        # Step 2 : Update task status to "In Progress"
        self.state["status"] = "In Progress"
        print(f"Task updated to : {self.state['task']} (Status : {self.state['status']})")

    @listen(start_task)
    def complete_task(self):
        # Step 3 : Update task status to "Completed"
        self.state["status"] = "Completed"
        print(f"Task Status updated : {self.state['status']}")
        print(f"Final Task State : {self.state}")


In [30]:
flow = TaskManagementFlow()
final_result = await flow.kickoff_async()

╭─────────────────────────────── Flow Execution ───────────────────────────────╮
│                                                                              │
│  Starting Flow Execution                                                     │
│  Name: TaskManagementFlow                                                    │
│  ID: a0c34ffc-8e1c-4e00-8ced-58ee0f89c45e                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: TaskManagementFlow
    ID: a0c34ffc-8e1c-4e00-8ced-58ee0f89c45e
└── 🧠 Starting Flow...

 Flow started with ID: a0c34ffc-8e1c-4e00-8ced-58ee0f89c45e
🌊 Flow: TaskManagementFlow
    ID: a0c34ffc-8e1c-4e00-8ced-58ee0f89c45e
├── 🧠 Starting Flow...
└── 🔄 Running: generate_task

Flow started. State ID : a0c34ffc-8e1c-4e00-8ced-58ee0f89c45e
Task gen

## Structured State

In [31]:
from pydantic import BaseModel

In [32]:
# Defining a structured state model
class TaskState(BaseModel):
    task: str = "None"
    status: str = "None"

In [37]:
from crewai.flow.flow import Flow, listen, start

class StructuredTaskFlow(Flow[TaskState]):

    @start()
    def generate_task(self):
        print(f"Flow started. State ID : {self.state.id}")
        self.state.task = "Develop a new API endpoint"
        self.state.status = "Pending"
        print(f"Task generated : {self.state.task} (Status : {self.state.status})")
    
    @listen(generate_task)
    def start_task(self):
        self.state.status = "In Progress"
        print(f"Task updated to : {self.state.task} (Status : {self.state.status})")

    @listen(start_task)
    def complete_task(self):
        self.state.status = "Completed"
        print(f"Task Status updated : {self.state.status}")
        print(f"Final Task State : {self.state}")
        

In [38]:
flow = StructuredTaskFlow()
final_result = await flow.kickoff_async()

╭─────────────────────────────── Flow Execution ───────────────────────────────╮
│                                                                              │
│  Starting Flow Execution                                                     │
│  Name: StructuredTaskFlow                                                    │
│  ID: e420f631-481c-410a-91a3-a738132ed42d                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: StructuredTaskFlow
    ID: e420f631-481c-410a-91a3-a738132ed42d
└── 🧠 Starting Flow...

 Flow started with ID: e420f631-481c-410a-91a3-a738132ed42d
🌊 Flow: StructuredTaskFlow
    ID: e420f631-481c-410a-91a3-a738132ed42d
├── 🧠 Starting Flow...
└── 🔄 Running: generate_task

Flow started. State ID : e420f631-481c-410a-91a3-a738132ed42d
Task gen

__we cannot dynamically add any new states during run-time__

## Conditional Flow Control

### OR conditional logic

In [40]:
from crewai.flow.flow import Flow, start, listen, or_

class SupportFlow(Flow):

    @start()
    def live_chat(self):
        return "Support request received via live chat"
    
    @start()
    def email_ticket_request(self):
        return "Support request received via email ticket"
    
    @listen(or_(live_chat, email_ticket_request))
    def log_request(self, request_source):
        print(f"Logging request : {request_source}")
        

In [42]:
flow = SupportFlow()
final_result = await flow.kickoff_async()    

╭─────────────────────────────── Flow Execution ───────────────────────────────╮
│                                                                              │
│  Starting Flow Execution                                                     │
│  Name: SupportFlow                                                           │
│  ID: 662d6f97-30d9-4e89-a461-fce7ac98f623                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: SupportFlow
    ID: 662d6f97-30d9-4e89-a461-fce7ac98f623
└── 🧠 Starting Flow...

 Flow started with ID: 662d6f97-30d9-4e89-a461-fce7ac98f623
🌊 Flow: SupportFlow
    ID: 662d6f97-30d9-4e89-a461-fce7ac98f623
├── 🧠 Starting Flow...
└── 🔄 Running: live_chat

🌊 Flow: SupportFlow
    ID: 662d6f97-30d9-4e89-a461-fce7ac98f623
├── Flow Method Step
└─

### AND condition logic

In [43]:
from crewai.flow.flow import Flow, and_, listen, start

class TicketEscalationFlow(Flow):

    @start()
    def user_confirms_issue(self):
        self.state["user_confirmation"] = True
        print("User confirmed they still need assistance.")

    @listen(user_confirms_issue)
    def agent_reviews_ticket(self):
        self.state["agent_review"] = True
        print("Support agent has reviewed the ticket.")

    @listen(and_(user_confirms_issue, agent_reviews_ticket))
    def escalate_ticket(self):
        print("Escalating ticket to Level 2 support!")

In [44]:
# Execute the flow
flow = TicketEscalationFlow()
final_result = await flow.kickoff_async()

╭─────────────────────────────── Flow Execution ───────────────────────────────╮
│                                                                              │
│  Starting Flow Execution                                                     │
│  Name: TicketEscalationFlow                                                  │
│  ID: 4e5696f3-c002-4e6a-b5ad-6f05144f30f7                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: TicketEscalationFlow
    ID: 4e5696f3-c002-4e6a-b5ad-6f05144f30f7
└── 🧠 Starting Flow...

 Flow started with ID: 4e5696f3-c002-4e6a-b5ad-6f05144f30f7
🌊 Flow: TicketEscalationFlow
    ID: 4e5696f3-c002-4e6a-b5ad-6f05144f30f7
├── 🧠 Starting Flow...
└── 🔄 Running: user_confirms_issue

User confirmed they still need assistance.
🌊 Flow: TicketEsc

## Router logic

In [51]:
import random
from crewai.flow.flow import Flow, listen, start, router

In [52]:
from pydantic import BaseModel

class TicketState(BaseModel):
    priority: str = "low"

In [56]:
class TicketRoutingFlow(Flow[TicketState]):
    @start()
    def classify_ticket(self):
        print("Classifying support ticket...")
        self.state.priority = random.choice(["high", "low"])
        print(f"Ticket classified as: {self.state.priority}")
    
    @router(classify_ticket)
    def route_ticket(self):
        if self.state.priority == "high":
            return "urgent_support"
        else:
            return "email_support"
    
    @listen("urgent_support")
    def assign_to_agent(self):
        print("Urgent ticket assigned to a live support agent!")
    
    @listen("email_support")
    def assign_to_team(self):
        print("Non-urgent ticket assigned to the email support team!")

In [58]:
# Execute the flow
flow = TicketRoutingFlow()
final_result = await flow.kickoff_async()

╭─────────────────────────────── Flow Execution ───────────────────────────────╮
│                                                                              │
│  Starting Flow Execution                                                     │
│  Name: TicketRoutingFlow                                                     │
│  ID: 06dc7e10-a302-41ab-aabc-69f110189ea4                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: TicketRoutingFlow
    ID: 06dc7e10-a302-41ab-aabc-69f110189ea4
└── 🧠 Starting Flow...

 Flow started with ID: 06dc7e10-a302-41ab-aabc-69f110189ea4
🌊 Flow: TicketRoutingFlow
    ID: 06dc7e10-a302-41ab-aabc-69f110189ea4
├── 🧠 Starting Flow...
└── 🔄 Running: classify_ticket

Classifying support ticket...
Ticket classified as: low
🌊 Flow: Ticket